In [1]:
from datetime import datetime as dt
import pandas as pd
import time
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [5]:
# url link from - https://www.redfin.com/news/data-center/
url_by_city = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz'

def extract_data(url):
    df = pd.read_csv(url, compression='gzip', sep='\t')    
    return df
    
df = extract_data(url_by_city)
df.head()


,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,state_code,property_type,property_type_id,median_sale_price,median_sale_price_mom,median_sale_price_yoy,median_list_price,median_list_price_mom,median_list_price_yoy,median_ppsf,median_ppsf_mom,median_ppsf_yoy,median_list_ppsf,median_list_ppsf_mom,median_list_ppsf_yoy,homes_sold,homes_sold_mom,homes_sold_yoy,pending_sales,pending_sales_mom,pending_sales_yoy,new_listings,new_listings_mom,new_listings_yoy,inventory,inventory_mom,inventory_yoy,months_of_supply,months_of_supply_mom,months_of_supply_yoy,median_dom,median_dom_mom,median_dom_yoy,avg_sale_to_list,avg_sale_to_list_mom,avg_sale_to_list_yoy,sold_above_list,sold_above_list_mom,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2019-06-01,2019-06-30,30,place,6,2714,f,"Carolina Shores, NC",Carolina Shores,North Carolina,NC,Townhouse,13,194375.0,0.023026,0.031714,189000.0,-0.099571,0.000000,127.542651,0.023026,0.031714,104.812500,-0.133167,-0.063039,7.0,0.000000,-0.363636,3.0,0.500000,-0.400000,4.0,-0.500000,-0.428571,27.0,-0.156250,0.421053,3.9,-0.7,2.2,224.0,-81.0,89.0,1.008397,0.009603,0.012746,0.714286,0.000000,0.350649,NaN,NaN,NaN,0.000000,-0.500000,-0.800000,"Myrtle Beach, SC",34820,2023-09-24 14:28:29
1,2021-04-01,2021-04-30,30,place,6,3223,f,"Charleston, UT",Charleston,Utah,UT,Single Family Residential,6,1825000.0,NaN,NaN,NaN,NaN,NaN,209.794229,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,37.0,NaN,NaN,1.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,"Heber, UT",25720,2023-09-24 14:28:29
2,2022-06-01,2022-06-30,30,place,6,23647,f,"Huntington, VA",Huntington,Virginia,VA,Single Family Residential,6,756000.0,0.281356,-0.184466,509500.0,-0.111595,0.110022,374.559732,-0.041127,0.174186,396.099843,0.012255,0.063222,2.0,-0.600000,1.000000,4.0,1.000000,0.000000,5.0,-0.166667,-0.166667,8.0,0.000000,0.600000,4.0,2.4,-1.0,14.0,10.0,10.0,1.019949,-0.027238,-0.051727,1.000000,0.200000,0.000000,0.5,0.125000,NaN,0.250000,-0.250000,-0.250000,"Washington, DC",47894,2023-09-24 14:28:29
3,2019-05-01,2019-05-31,30,place,6,5832,f,"Everett, WA",Everett,Washington,WA,Townhouse,13,330000.0,-0.083333,-0.017857,399999.0,0.000060,0.028275,231.254380,-0.024982,-0.037982,256.849315,-0.011699,0.067470,13.0,-0.133333,-0.606061,17.0,0.214286,-0.433333,21.0,0.166667,-0.432432,20.0,0.176471,0.333333,1.5,0.4,1.0,9.0,0.0,1.0,0.999617,-0.003677,-0.023351,0.384615,0.051282,-0.100233,0.3,0.005882,0.033333,0.588235,-0.268908,-0.278431,"Seattle, WA",42644,2023-09-24 14:28:29
4,2014-12-01,2014-12-31,30,place,6,4302,f,"Daytona Beach, FL",Daytona Beach,Florida,FL,All Residential,-1,99000.0,-0.120000,0.015385,117900.0,0.026110,0.071818,75.729927,-0.075651,-0.013181,84.244080,-0.083734,-0.123988,111.0,0.247191,0.018349,46.0,0.533333,0.437500,81.0,-0.190000,0.051948,588.0,-0.036066,-0.001698,5.3,-1.6,-0.1,127.0,-58.0,-14.0,0.921752,-0.012893,0.002645,0.063063,-0.049296,-0.010331,NaN,NaN,NaN,0.130435,-0.002899,0.067935,"Deltona, FL",19660,2023-09-24 14:28:29


In [6]:
print('Num of rows:', len(df))
print('Num of cols:', len(df.columns))

Num of rows: 4960580
Num of cols: 58


In [11]:
# url link from - https://www.redfin.com/news/data-center/
url_by_city = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz'

def lambda_handler(url):
    df = pd.read_csv(url, compression='gzip', sep='\t')
   
   
     # Remove commas from the 'city' column
    df['city'] = df['city'].str.replace(',', '')
    cols = ['period_begin','period_end','period_duration', 'region_type', 'region_type_id', 'table_id',
    'is_seasonally_adjusted', 'city', 'state', 'state_code', 'property_type', 'property_type_id',
    'median_sale_price', 'median_list_price', 'median_ppsf', 'median_list_ppsf', 'homes_sold',
    'inventory', 'months_of_supply', 'median_dom', 'avg_sale_to_list', 'sold_above_list', 'parent_metro_region_metro_code', 
    'last_updated']
    df = df[cols]
    
    df = df.dropna()
    
    #let's change the period_begin and period_end to date time object and extract years and month.
    df['period_begin'] = pd.to_datetime(df['period_begin'])
    df['period_end'] = pd.to_datetime(df['period_end'])
    
    df["period_begin_in_years"] = df['period_begin'].dt.year
    df["period_end_in_years"] = df['period_end'].dt.year

    df["period_begin_in_months"] = df['period_begin'].dt.month
    df["period_end_in_months"] = df['period_end'].dt.month
    
    #let's map the month number to their respective month name.
    month_dict = {
        "period_begin_in_months": {
            1 : "Jan",
             2 :  "Feb",
             3: "Mar",
             4: "Apr",
             5: "May",
             6: "Jun",
             7: "Jul",
             8: "Aug",
             9: "Sep",
             10: "Oct",
             11: "Nov",
             12: "Dec",
        },
        "period_end_in_months": {
            1 : "Jan",
             2 :  "Feb",
             3: "Mar",
             4: "Apr",
             5: "May",
             6: "Jun",
             7: "Jul",
             8: "Aug",
             9: "Sep",
             10: "Oct",
             11: "Nov",
             12: "Dec"
        }}
    df = df.replace(month_dict)
    print('Num of rows:', len(df))
    print('Num of cols:', len(df.columns))
    return df

In [12]:
df = lambda_handler(url_by_city)
df.head()

Num of rows: 3932161
Num of cols: 28


,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,city,state,state_code,property_type,property_type_id,median_sale_price,median_list_price,median_ppsf,median_list_ppsf,homes_sold,inventory,months_of_supply,median_dom,avg_sale_to_list,sold_above_list,parent_metro_region_metro_code,last_updated,period_begin_in_years,period_end_in_years,period_begin_in_months,period_end_in_months
0,2019-06-01,2019-06-30,30,place,6,2714,f,Carolina Shores,North Carolina,NC,Townhouse,13,194375.0,189000.0,127.542651,104.812500,7.0,27.0,3.9,224.0,1.008397,0.714286,34820,2023-09-24 14:28:29,2019,2019,Jun,Jun
2,2022-06-01,2022-06-30,30,place,6,23647,f,Huntington,Virginia,VA,Single Family Residential,6,756000.0,509500.0,374.559732,396.099843,2.0,8.0,4.0,14.0,1.019949,1.000000,47894,2023-09-24 14:28:29,2022,2022,Jun,Jun
3,2019-05-01,2019-05-31,30,place,6,5832,f,Everett,Washington,WA,Townhouse,13,330000.0,399999.0,231.254380,256.849315,13.0,20.0,1.5,9.0,0.999617,0.384615,42644,2023-09-24 14:28:29,2019,2019,May,May
4,2014-12-01,2014-12-31,30,place,6,4302,f,Daytona Beach,Florida,FL,All Residential,-1,99000.0,117900.0,75.729927,84.244080,111.0,588.0,5.3,127.0,0.921752,0.063063,19660,2023-09-24 14:28:29,2014,2014,Dec,Dec
5,2016-06-01,2016-06-30,30,place,6,13436,f,Marysville,Michigan,MI,All Residential,-1,116000.0,133900.0,93.243790,99.784816,22.0,44.0,2.0,14.0,0.984224,0.227273,47664,2023-09-24 14:28:29,2016,2016,Jun,Jun


In [13]:
#number of years in our dataset
# sorted(df["period_begin_in_years"].unique())

[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [ ]:
# download file
# df.to_csv('real_estate_w209_byCity.csv', index=False)